The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install -U mxnet-cu101==1.7.0
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l


# 自动并行
:label:`sec_auto_para`

深度学习框架（例如，MxNet 和 PyTorch）会在后端自动构建计算图。利用计算图，系统可以了解所有依赖关系，并且可以选择性地并行执行多个不相互依赖的任务以提高速度。例如， :numref:`sec_async` 中的 :numref:`fig_asyncgraph` 独立初始化两个变量。因此，系统可以选择并行执行它们。 

通常，单个操作符将使用所有cpu或单个GPU上的所有计算资源。例如，`dot`操作符将使用所有CPU上的所有核心（和线程），即使一台机器上有多个CPU处理器。这同样适用于单个GPU。因此，对于单设备计算机来说，并行化并不是很有用。有了多个设备，并行化就重要了。虽然并行化通常在多个GPU之间，但添加本地CPU将略微提高性能。例如，请参阅 :cite:`Hadjis.Zhang.Mitliagkas.ea.2016` ，它着重于训练结合GPU和CPU的计算机视觉模型。借助自动并行化框架的便利性，我们可以在几行Python代码中实现相同的目标。更广泛地说，我们对自动并行计算的讨论集中在使用CPU和GPU的并行计算，以及计算和通信的并行化。

请注意，我们至少需要两个GPU来运行本节中的实验。


In [1]:
from mxnet import np, npx
from d2l import mxnet as d2l

npx.set_np()

## 基于GPU的并行计算

让我们从定义一个参考工作负载用于测试开始：下面的`run`函数使用分配到两个变量（`x_gpu1`和`x_gpu2`）的数据在我们选择的设备上执行10次矩阵-矩阵乘法。


In [2]:
devices = d2l.try_all_gpus()

def run(x):
    return [x.dot(x) for _ in range(50)]

x_gpu1 = np.random.uniform(size=(4000, 4000), ctx=devices[0])
x_gpu2 = np.random.uniform(size=(4000, 4000), ctx=devices[1])

现在我们将函数应用于数据。为了确保缓存在结果中不起作用，我们通过在测量之前对其中任何一个设备执行一次传递来预热设备。


In [3]:
run(x_gpu1)  # 预热设备
run(x_gpu2)
npx.waitall()

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    npx.waitall()

with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    npx.waitall()

GPU1 time: 0.4820 sec


GPU2 time: 0.4920 sec


如果我们删除两个任务之间的`waitall`语句，系统就可以在两个设备上自动并行计算。


In [4]:
with d2l.Benchmark('GPU1 & GPU2'):
    run(x_gpu1)
    run(x_gpu2)
    npx.waitall()

GPU1 & GPU2: 0.4925 sec


在上述情况下，总执行时间小于其部分的总和，因为深度学习框架自动调度两个GPU设备上的计算，而不需要用户编写复杂的代码。

## 并行计算与通信

在许多情况下，我们需要在不同的设备之间移动数据，比如在CPU和GPU之间，或者在不同的GPU之间。例如，当我们想要执行分布式优化时，需要聚合多个加速卡上的梯度时，就会遇到这种情况。让我们通过在GPU上计算，然后将结果复制回CPU来模拟这个过程。


In [5]:
def copy_to_cpu(x):
    return [y.copyto(npx.cpu()) for y in x]

with d2l.Benchmark('Run on GPU1'):
    y = run(x_gpu1)
    npx.waitall()

with d2l.Benchmark('Copy to CPU'):
    y_cpu = copy_to_cpu(y)
    npx.waitall()

Run on GPU1: 0.5117 sec


Copy to CPU: 2.3707 sec


这有点低效。请注意，我们可能已经开始将`y`的部分复制到CPU，而列表的其余部分仍在计算中。这种情况会发生，例如，当我们计算一个小批量的梯度时。某些参数的梯度将比其他参数的梯度更早可用。因此，在GPU仍在运行时开始使用PCI-Express总线带宽对我们是有利的。删除这两个部分之间的`waitall`允许我们模拟这种情况。


In [6]:
with d2l.Benchmark('在GPU1上运行并复制到CPU'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y)
    npx.waitall()

在GPU1上运行并复制到CPU: 2.5541 sec


两个操作所需的总时间少于它们各部分的总和。请注意，此任务与并行计算不同，因为它使用不同的资源：CPU和GPU之间的总线。事实上，我们可以在两个设备上同时进行计算和通信。如上所述，计算和通信之间存在依赖关系：必须先计算`y[i]`，然后才能将其复制到CPU。幸运的是，系统可以在计算`y[i]`的同时复制`y[i-1]`，以减少总的运行时间。

最后，我们给出了一个简单的两层多层感知机在CPU和两个GPU上训练时的计算图及其依赖关系的例子，如 :numref:`fig_twogpu` 所示。手动调度由此产生的并行程序将是相当痛苦的。这就是基于图的计算后端进行优化的优势所在。

![一个CPU和两个GPU上的两层多层感知机的计算图及其依赖关系。](http://d2l.ai/_images/twogpu.svg)
:label:`fig_twogpu`

## 小结

* 现代系统有多种设备，如多个GPU和CPU。它们可以并行、异步地使用。
* 现代系统还拥有各种通信资源，如PCI Express、存储（通常是固态驱动器或通过网络）和网络带宽。它们可以并行使用，达到最高效率。
* 后端可以通过自动并行计算和通信来提高性能。

## 练习

1. 在本节定义的`run`函数中执行了八个操作。它们之间没有依赖关系。设计一个实验，看看深度学习框架是否会自动并行执行它们。
1. 当如果单个操作符的工作量足够小，即使在单个CPU或GPU上，并行化也会有所帮助。设计一个实验来验证这一点。
1. 设计一个实验，在CPU、GPU上使用并行计算，并在两个设备之间进行通信。
1. 使用诸如NVIDIA的[Nsight](https://developer.nvidia.com/nsight-compute-2019_5)之类的调试器来验证你的代码是否有效。
1. 设计包含更复杂数据依赖关系的计算任务，并运行实验，以查看是否可以在提高性能的同时获得正确的结果。


[Discussions](https://discuss.d2l.ai/t/2795)
